In [1]:
from abakit.lib.Controllers.UrlController import UrlController
from abakit.lib.Controllers.ScanRunController import ScanRunController
from abakit.lib.annotation_layer import AnnotationLayer
from abakit.atlas.VolumeMaker import VolumeMaker
from abakit.atlas.NgSegmentMaker import NgConverter
import os 
import numpy as np
import sys
import argparse
import json
def contours_to_volume(url_id, volume_id):
    controller = UrlController()
    urlModel = controller.get_urlModel(url_id)
    state_json = json.loads(urlModel.url)
    layers = state_json['layers']
    for layeri in layers:
        if layeri['type'] == 'annotation':
            layer = AnnotationLayer(layeri)
            volume = layer.get_annotation_with_id(volume_id)
            if volume is not None:
                break
    animal = urlModel.get_animal()
    folder_name = make_volumes(volume, animal)
    segmentation_save_folder = f"precomputed://https://activebrainatlas.ucsd.edu/data/structures/{folder_name}"
    return segmentation_save_folder

def downsample_contours(contours, downsample_factor):
    values = [i/downsample_factor for i in contours.values()]
    return dict(zip(contours.keys(), values))

def get_scale(animal, downsample_factor):
    controller = ScanRunController()
    scan_run = controller.get_scan_run(animal)
    res = scan_run.resolution
    return [downsample_factor*res*1000, downsample_factor*res*1000, scan_run.zresolution*1000]

def make_volumes(volume, animal='DK55', downsample_factor=20):    
    vmaker = VolumeMaker()
    structure,contours = volume.get_volume_name_and_contours()
    downsampled_contours = downsample_contours(contours,downsample_factor)
    vmaker.set_aligned_contours({structure:downsampled_contours})
    vmaker.compute_origins_and_volumes_for_all_segments(interpolate=1)
    volume = (vmaker.volumes[structure]).astype(np.uint8)
    offset = list(vmaker.origins[structure])
    folder_name = f'{animal}_{structure}'
    path = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/structures'
    output_dir = os.path.join(path,folder_name)
    scale = get_scale(animal,downsample_factor)
    maker = NgConverter(volume = volume,scales = scale,offset=offset)
    maker.create_neuroglancer_files(output_dir,segment_properties=[(1,structure)])
    return folder_name

if __name__=='__main__':
    # parser = argparse.ArgumentParser(description="url and volume id")
    # parser.add_argument("--url", help="url id", required=True,type=int)
    # parser.add_argument("--volume", help="volume id", required=False, default=1,type=str)
    # args = parser.parse_args()
    url = contours_to_volume(312,'2b9c285bdf34b5df5378b572748da0512238ab31')
    # url = contours_to_volume(args.url,args.volume)
    sys.stdout.write(url)
    sys.stdout.flush()
    sys.exit(0)

Uploading:  67%|██████▋   | 12/18 [00:00<00:00, 42.79it/s]

/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/structures/DK60_7N_R


Tasks:   0%|          | 0/2 [00:00<?, ?it/s]

Volume Bounds:  Bbox([1947, 1138, 505],[2083, 1312, 597], dtype=int32)
Selected ROI:   Bbox([1947, 1138, 505],[2083, 1312, 597], dtype=int32)


Tasks:   0%|          | 0/2 [00:00<?, ?it/s]


ScaleUnavailableError: MIP 1 not found. 1 available: <6500.0,6500.0,20000.0>

In [9]:
[res*1000, res*1000, scan_run.zresolution*1000]

[325.0, 325.0, 20000.0]

In [8]:
controller = ScanRunController()
scan_run = controller.get_scan_run(animal)
res = scan_run.resolution
[downsample_factor*res*1000, downsample_factor*res*1000, scan_run.zresolution*1000]

[6500.0, 6500.0, 20000.0]

In [16]:
url_id, volume_id = 312,'2b9c285bdf34b5df5378b572748da0512238ab31'
controller = UrlController()
urlModel = controller.get_urlModel(url_id)
state_json = json.loads(urlModel.url)
layers = state_json['layers']
for layeri in layers:
    if layeri['type'] == 'annotation':
        layer = AnnotationLayer(layeri)
        volume = layer.get_annotation_with_id(volume_id)
        if volume is not None:
            break
animal = urlModel.get_animal()
downsample_factor=20
vmaker = VolumeMaker()
structure,contours = volume.get_volume_name_and_contours()
downsampled_contours = downsample_contours(contours,downsample_factor)
vmaker.set_aligned_contours({structure:downsampled_contours})
vmaker.compute_origins_and_volumes_for_all_segments(interpolate=1)
volume = (vmaker.volumes[structure]).astype(np.uint8)
offset = list(vmaker.origins[structure])
folder_name = f'{animal}_{structure}'
path = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/structures'
output_dir = os.path.join(path,folder_name)
scale = get_scale(animal,downsample_factor)
maker = NgConverter(volume = volume,scales = scale,offset=offset)
segment_properties=[(0,structure)]
maker.reset_output_path(output_dir)
maker.init_precomputed(output_dir)
maker.add_segment_properties(segment_properties)
maker.add_segmentation_mesh()
segmentation_save_folder = f"precomputed://https://activebrainatlas.ucsd.edu/data/structures/{folder_name}"

Uploading:  83%|████████▎ | 15/18 [00:00<00:00, 74.81it/s]

/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/structures/DK60_7N_R


Tasks: 100%|██████████| 999/999 [00:01<00:00, 602.84it/s]


In [6]:
segment_properties

[(0, '7N_R')]

In [11]:
maker.precomputed_vol.scale

AttributeError: 'NoneType' object has no attribute 'scale'

In [15]:
path

'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/structures'

In [16]:
self.precomputed_vol = CloudVolume(f'file://{path}', mip=0, info=info, compress=True, progress=True)


In [17]:
from cloudvolume import CloudVolume
self = maker
info = CloudVolume.create_new_info(
        num_channels = self.volume.shape[3] if len(self.volume.shape) > 3 else 1,
        layer_type = self.layer_type,
        data_type = str(self.volume.dtype),  # Channel images might be 'uint8'
        encoding = 'raw',                    # raw, jpeg, compressed_segmentation, fpzip, kempressed
        resolution = self.scales,            # Voxel scaling, units are in nanometers
        voxel_offset = self.offset -np.array([1,1,1]),          # x,y,z offset in voxels from the origin
        chunk_size = [64,64,64],           # units are voxels
        volume_size = self.volume.shape[:3], # e.g. a cubic millimeter dataset
    )
self.precomputed_vol = CloudVolume(f'file://{path}', mip=0, info=info, compress=True, progress=True)
self.precomputed_vol.commit_info()
self.precomputed_vol[:, :, :] = self.volume

Uploading: 100%|██████████| 18/18 [00:00<00:00, 71.07it/s]
